In [1]:
import scipy.io
import h5py
import numpy as np
from functions import matlab2python, call_remote_file_ssh
from timeit import default_timer as timer

In [2]:
# print the available fields 
fields_allEv = matlab2python(0, 'ct20_test', 'allEv', None)
fields_perSyn = matlab2python(0, 'ct20_test', 'perSyn', None)

print('fields_allEv',fields_allEv)
print('total', len(fields_allEv))
print('*****************************')
print('fields_perSyn',fields_perSyn)
print('total', len(fields_perSyn))


Executing matlab2python with arguments: actionFlag=0, file_name='ct20_test', section='allEv', fields2call=None, filename2save='results.csv'


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/media/data/analysis/summary/ct20_test.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#use the desired fields
#matlab2python(actionFlag, file_name, section ,fields2call, filename2save = 'results.csv', file_path='/media/data/analysis/summary'):
data_allEv = matlab2python(1, 'ct20_test', 'allEv', ['synID','x','y', 'timeFromLast'], '/home/gerard/nextcloud/analysis/synaptic_computation/results/results.csv')
print(data_allEv.shape)

OPENING : /media/data/analysis/summary/ct20_test.mat
0 synID
1 x
2 y
3 timeFromLast
file saved as csv
(68238, 4)


In [ ]:
print(data_allEv[0])
print(data_allEv[0,0])
print(data_allEv[:,0])

[5.85060000e+04 2.10154416e+01 2.00624993e+01            nan]
58506.0
[5.85060000e+04 5.85060000e+04 5.85060000e+04 ... 1.11021137e+09
 1.11021137e+09 1.11021137e+09]


In [ ]:
#use ALL the fields
data_allEv_all = matlab2python(1, 'ct20_test', 'allEv', fields_allEv)
print(data_allEv_all.shape)

/media/data/analysis/summary/ct20_test.mat
OPENING : /media/data/analysis/summary/ct20_test.mat
0 A
1 A_pstd
2 SDa
3 SDb
4 SDc
5 SDpreF
6 asynch2nd
7 asynchAlloc
8 c
9 c_pstd
10 df
11 dfCrit
12 dist2Reclust
13 dist2centFAll
14 dist2centFAsynch


KeyboardInterrupt: 

In [ ]:
data_perSyn_all = matlab2python(1, 'ct20_test', 'perSyn', fields_perSyn)
print(data_perSyn_all.shape)

OPENING : /home/gerard/nextcloud/analysis/localization/summary/ct20_test.mat
0 Amp
1 NumAsynch2
2 NumAsynchAllo
3 NumSynch
4 NumclustAll
5 NumclustAsynch
6 NumclustSynch
7 NumclustSynch2nd
8 areaAll
9 areaAsynch
10 areaSynch
11 areaSynch2nd
12 bckgrd
13 columnNum
14 columnNumCorr
15 df_tail
16 rowNum
17 rowNumCorr
18 sigma
19 snr
20 synID
long field, not included  trace
(944, 21)


In [ ]:
start = timer()
# Replace 'file_path' with the path to your MATLAB file.
file_path = '/media/data/nextcloud/analysis/summary/ct20_test.mat' #/home/gerard/analysis/from_matlab/summary/ct20_test.mat'
fields2call = ['synID','x','y', 'timeFromLast']
file_name = '/ct20_test/allEv/'

# Open the MATLAB file using h5py
with h5py.File(file_path, 'r') as file:
    # Access the 'allEv' dataset
    allEv_data = np.array([])#[]

    for idx, name in enumerate(fields2call):

        allEv_dataset = file[file_name + name]

        # Convert the references to strings
        allEv_references = [ref.item() for ref in allEv_dataset]

        carrier =  []
        for ref in allEv_references:
            item = file[ref]
            carrier.append(item[:])
        carrier = np.squeeze(np.array(carrier))

        if allEv_data.size == 0:
           allEv_data = np.append(allEv_data, carrier)

        else:
            allEv_data = np.column_stack((allEv_data, carrier))
        
end = timer()
print(end - start)

24.65767366199725


In [ ]:
print(type(allEv_data))
print(np.shape(allEv_data))
print(allEv_data[0,0], allEv_data[0,1], allEv_data[0,2])

<class 'numpy.ndarray'>
(68238, 3)
58506.0 21.01544156606587 20.0624993068723


In [ ]:
ssh_credentials = {
    'hostname': '192.168.0.143',
    'port': 22,
    'username': 'gerard',
    'password': 'naranja8712',
    'file_path': '/media/data/analysis/summary'
}

In [ ]:
call_remote_file_ssh('ct20_test', 'perSyn', ssh_credentials)

AuthenticationException: Authentication failed.